In [9]:
conda install -c conda-forge spacy

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.

# >>>>>>>>>>>>>>>>>>>>>> ERROR REPORT <<<<<<<<<<<<<<<<<<<<<<

    Traceback (most recent call last):
      File "/opt/anaconda3/lib/python3.7/site-packages/conda/exceptions.py", line 1079, in __call__
        return func(*args, **kwargs)
      File "/opt/anaconda3/lib/python3.7/site-packages/conda/cli/main.py", line 84, in _main
        exit_code = do_call(args, p)
      File "/opt/anaconda3/lib/python3.7/site-packages/conda/cli/conda_argparse.py", line 83, in do_call
        return getattr(module, func_name)(args, parser)
      File "/opt/anaconda3/lib/python3.7/site-packages/conda/cli/main_install.py", line 20, in execute
        install(args, parser, 'install')
      File "/opt/anaconda3/lib/python3.7/site-packages/conda/cli/install.py", line 116, in install
        if context.use_only_tar_bz2:
      File "/opt/anaconda3/lib/python3.7/site-pa

In [1]:
# New Stuff
from torchtext.data import Field, TabularDataset, BucketIterator, LabelField
import spacy
import torch

torch.backends.cudnn.deterministic = True

# spacy_en = spacy.load('en')

#     return [token.text for token in spacy_en.tokenizer(text)]

# title = Field(sequential=True, use_vocab=True, tokenize='spacy', lower=True)
text = Field(sequential=True, use_vocab=True, tokenize='spacy', include_lengths = True, lower=True)
label = LabelField(dtype=torch.float)

fields = {'text': ('text', text), 'label': ('label', label)}

In [25]:
train_data = TabularDataset.splits(
    path='../data',
    train='news.csv',
    # validation='news.csv',
    # test='news.csv',
    format='csv',
    fields=fields)[0]
print("Num of training: ", len(train_data))
# print("Num of validation: ", len(validation_data))
# print("Num of testing: ", len(test_data))

Num of training:  6335


In [26]:
train_data, validation_data = train_data.split(split_ratio=0.65)
validation_data, test_data = validation_data.split(split_ratio=0.5)
print("Num of training: ", len(train_data))
print("Num of validation: ", len(validation_data))
print("Num of testing: ", len(test_data))

Num of training:  4118
Num of validation:  1108
Num of testing:  1109


In [30]:
MAX_VOCAB_SIZE = 25_000

text.build_vocab(train_data, max_size = MAX_VOCAB_SIZE, vectors = "glove.6B.100d", 
unk_init = torch.Tensor.normal_)
label.build_vocab(train_data)

In [32]:
BATCH_SIZE = 64

device = torch.device('cpu')

train_iterator, validation_iterator, test_iterator = BucketIterator.splits(
    (train_data, validation_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [33]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
    bidirectional, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers = n_layers, bidirectional=bidirectional, dropout = dropout )
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):
        #text = [sent len, batch size]
        embedded = self.dropout(self.embedding(text))
        # embedded = [sent len, batch size, emb dim]
        
        # pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        packed_output, (hidden, cell) = self.rnn(packed_embedded)

        # unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout

        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))

        #hidden = [batch size, hid dim * num directions]

        return self.fc(hidden)



In [34]:
INPUT_DIM = len(text.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = text.vocab.stoi[text.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)


In [35]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,810,857 trainable parameters


In [36]:
pretrained_embeddings = text.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [37]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0497,  0.3701, -0.4061,  ..., -1.0511, -0.1554, -1.0198],
        [ 0.4551, -1.4843, -0.2436,  ...,  0.8461,  0.0814,  0.1467],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.1423, -0.0908,  0.1016,  ...,  0.2297,  0.5268, -0.8832],
        [-0.1959,  0.1677, -0.8102,  ...,  0.1850, -0.4534,  0.3274],
        [-0.3076, -0.7861,  1.2381,  ..., -0.3210,  0.9117, -0.3930]])

In [39]:
UNK_IDX = text.vocab.stoi[text.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.1423, -0.0908,  0.1016,  ...,  0.2297,  0.5268, -0.8832],
        [-0.1959,  0.1677, -0.8102,  ...,  0.1850, -0.4534,  0.3274],
        [-0.3076, -0.7861,  1.2381,  ..., -0.3210,  0.9117, -0.3930]])


In [40]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())